In [ ]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.examples.tutorials.mnist import input_data
from skimage import transform
import numpy as np
import torch.optim as optim
# import tensorflow_datasets as tfds

mnist = input_data.read_data_sets("MNIST_data", one_hot=False)

In [5]:
class CNN(nn.Module):
    def __init__(self, n_classes):
        super(CNN, self).__init__()
        # conv layers: (in_channel size, out_channels size, kernel_size, stride, padding)
        self.conv1 = nn.Conv2d(1, 32, 5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(32, 16, 5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(16, 8, 5, stride=1, padding=2)

        # max pooling (kernel_size, stride)
        self.pool = nn.MaxPool2d(2, 2)

        # fully conected layers:
        self.layer1 = nn.Linear(4*4*8, 64)
        self.layer2 = nn.Linear(64, 64)
        self.layer3 = nn.Linear(64, n_classes)

    def forward(self, x, training=True):
        # the autoencoder has 3 con layers and 3 deconv layers (transposed conv). All layers but the last have ReLu
        # activation function
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 4 * 4 * 8)
        x = F.relu(self.layer1(x))
        x = F.dropout(x, 0.5, training=training)
        x = F.relu(self.layer2(x))
        x = F.dropout(x, 0.5, training=training)
        x = self.layer3(x)
        return x

    def predict(self, x):
        # a function to predict the labels of a batch of inputs
        x = F.softmax(self.forward(x, training=False))
        return x

    def accuracy(self, x, y):
        # a function to calculate the accuracy of label prediction for a batch of inputs
        #   x: a batch of inputs
        #   y: the true labels associated with x
        prediction = self.predict(x)
        maxs, indices = torch.max(prediction, 1)
        acc = 100 * torch.sum(torch.eq(indices.float(), y.float()).float())/y.size()[0]
        return acc.cpu().data[0]


In [6]:
model = CNN(10)
model.cuda()

CNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(32, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(16, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (layer1): Linear(in_features=128, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=10, bias=True)
)

In [7]:
state = torch.load('cnn.pth')
model.load_state_dict(state)

<All keys matched successfully>

In [9]:
modules = []
for name, module in model.named_modules():
    modules.append(name)
print(modules) 

['', 'conv1', 'conv2', 'conv3', 'pool', 'layer1', 'layer2', 'layer3']


In [70]:
getattr(model, modules[1])

tensor([[[[-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          ...,
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003],
          [-0.0003, -0.0003, -0.0003,  ..., -0.0003, -0.0003, -0.0003]],

         [[ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312],
          [ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312],
          [ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312],
          ...,
          [ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312],
          [ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312],
          [ 0.1312,  0.1312,  0.1312,  ...,  0.1312,  0.1312,  0.1312]],

         [[-0.0007, -0.0007, -0.0007,  ..., -0.0007, -0.0007, -0.0007],
          [-0.0007, -0.0007, -

In [34]:
batch_size = 500   # Number of samples in each batch
epoch_num = 20      # Number of epochs to train the network
lr = 0.0005        # Learning rate

def next_batch(train=True):
    # A function to read the next batch of MNIST images and labels
    # Args:
    #   train: a boolean array, if True it will return the next train batch, otherwise the next test batch
    # Returns:
    #   batch_img: a pytorch Variable of size [batch_size, 1, 32, 32].
    #   batch_label: a pytorch Variable of size [batch_size, ].

    if train:
        batch_img, batch_label = mnist.train.next_batch(batch_size)
    else:
        batch_img, batch_label = mnist.test.next_batch(batch_size)

    # reshape the sample to a batch of images in pytorch order (batch, channels, height, width)
    batch_img = batch_img.reshape((-1, 1, 28, 28))

    # resize the images to (32, 32)
    resized_imgs = np.zeros((batch_img.shape[0], 1, 32, 32))
    for i in range(batch_img.shape[0]):
        resized_imgs[i, 0, ...] = transform.resize(batch_img[i, 0,...], (32, 32))

    batch_label = torch.from_numpy(batch_label).long()  # convert the numpy array into torch tensor
    batch_label = Variable(batch_label).cuda()          # create a torch variable and transfer it into GPU

    resized_imgs = torch.from_numpy(resized_imgs).float()     # convert the numpy array into torch tensor
    resized_imgs = Variable(resized_imgs).cuda()              # create a torch variable and transfer it into GPU
    return resized_imgs, batch_label

In [35]:
features, labels = next_batch(train=False)

np.argmax(model.predict(features).cpu().detach().numpy(), axis=1)

/home/saqib/.local/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


array([6, 6, 4, 1, 6, 0, 5, 9, 3, 7, 6, 1, 3, 3, 5, 2, 1, 8, 3, 8, 9, 1,
       8, 5, 1, 6, 9, 5, 7, 7, 5, 8, 1, 7, 0, 8, 0, 4, 7, 5, 8, 7, 2, 1,
       5, 4, 0, 3, 7, 7, 8, 7, 3, 7, 6, 5, 2, 2, 9, 8, 9, 0, 2, 9, 0, 7,
       9, 7, 9, 7, 3, 4, 4, 5, 0, 5, 9, 9, 9, 7, 9, 7, 7, 6, 0, 0, 9, 3,
       0, 7, 0, 3, 6, 1, 9, 5, 4, 0, 3, 1, 8, 9, 4, 8, 0, 1, 7, 2, 8, 7,
       0, 2, 2, 7, 6, 1, 1, 6, 6, 1, 3, 9, 2, 8, 9, 8, 9, 9, 2, 8, 3, 8,
       9, 4, 5, 5, 4, 7, 6, 3, 9, 5, 1, 7, 6, 5, 0, 8, 5, 9, 6, 3, 5, 3,
       0, 9, 5, 7, 5, 5, 1, 4, 9, 9, 3, 3, 0, 2, 8, 5, 7, 5, 2, 2, 1, 6,
       8, 4, 7, 0, 9, 9, 5, 2, 3, 4, 7, 0, 1, 2, 5, 4, 1, 3, 4, 3, 9, 1,
       6, 0, 7, 2, 7, 1, 6, 0, 7, 9, 8, 4, 6, 6, 0, 0, 2, 4, 6, 2, 7, 4,
       0, 4, 5, 5, 9, 0, 0, 6, 1, 0, 9, 4, 5, 6, 7, 6, 8, 9, 1, 1, 1, 8,
       4, 2, 2, 8, 2, 3, 6, 9, 0, 4, 7, 9, 8, 3, 4, 3, 4, 3, 3, 7, 7, 2,
       1, 6, 4, 8, 4, 3, 1, 8, 8, 1, 1, 1, 0, 7, 3, 2, 4, 8, 5, 0, 4, 3,
       5, 1, 9, 4, 7, 7, 0, 5, 5, 8, 9, 0, 4, 2, 8,

In [ ]:
weights = model.conv1.weight
weights

In [ ]:
model.conv1.weight = weights
model.conv1.weight

In [64]:
features.shape

torch.Size([500, 1, 32, 32])

In [68]:
model.conv1.forward(features).shape

torch.Size([500, 32, 32, 32])

In [45]:
dir(model.conv1)

['__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_modules',
 '_named_members',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_tracing_name',
 '_version',
 'add_module',
 'apply',
 'bias',
 'buffers',
 'children',
 'conv2d_forward',
 'cpu',
 'cuda',
 'dilation',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'groups',
 'ha